In [ ]:
import numpy as np
import pickle
import json
import ast

file_paths = [
    './data/nasbench201/pickle/desktop-cpu-core-i7-7820x-fp32.pickle',
    './data/nasbench201/pickle/desktop-gpu-gtx-1080ti-fp32.pickle',
    './data/nasbench201/pickle/desktop-gpu-gtx-1080ti-large.pickle',
    './data/nasbench201/pickle/embedded-gpu-jetson-nono-fp16.pickle',
    './data/nasbench201/pickle/embedded-tpu-edgetpu-large.pickle',
    './data/nasbench201/pickle/embeeded-tpu-edgetpu-int8.pickle',
    './data/nasbench201/pickle/mobile-cpu-snapdragon-450-contex-a53-int8.pickle',
    './data/nasbench201/pickle/mobile-cpu-snapdragon-675-kryo-460-int8.pickle',
    './data/nasbench201/pickle/mobile-cpu-snapdragon-855-kryo-485-int8.pickle',
    './data/nasbench201/pickle/mobile-dsp-snapdragon-675-hexagon-685-int8.pickle',
    './data/nasbench201/pickle/mobile-dsp-snapdragon-855-hexagon-690-int8.pickle',
    './data/nasbench201/pickle/mobile-gpu-snapdragon-450-adreno-506-int8.pickle',
    './data/nasbench201/pickle/mobile-gpu-snapdragon-675-adren0-612-int8.pickle',
    './data/nasbench201/pickle/mobile-gpu-snapdragon-855-adren0-640-int8.pickle'
]


key_map = {}
idx = 0
first = True
all_df = np.zeros((len(file_paths), 15284, 6))
all_y = np.zeros((len(file_paths), 15284, 1))
for i, file_path in enumerate(file_paths):
    if first:
        idx = 0
        try:
            with open(file_path,'rb') as file:
                loaded_data = pickle.load(file)
                for key, value in loaded_data.items():
                    if key not in key_map:
                        all_df[i, idx] = key
                        key_map[key] = idx
                        all_y[i, idx] = value
                        idx += 1
        except Exception as e:
                print(f"Error loading {file_path}: {e}")
        first = False
    else:
        try:
            with open(file_path,'rb') as file:
                loaded_data = pickle.load(file)
                for key, value in loaded_data.items():
                    all_df[i, key_map[key]] = key 
                    all_y[i, key_map[key]] = value
        except Exception as e:
            print(f"Error loading {file_path}: {e}")

import os
for i, file_path in enumerate(file_paths):
    x = all_df[i]
    y = all_y[i]
    data = np.concatenate((x, y), axis=1)
    os.makedirs('./data/nasbench201/pkl', exist_ok=True)
    output_file_path = f'./data/nasbench201/pkl/{file_path.split("/")[-1].replace(".pickle", ".pkl")}'

    with open(output_file_path,'wb') as output_file:
        pickle.dump(data,output_file)

In [11]:
from data_provider.data_center import DataModule
from run_train import get_experiment_name
from utils.exp_config import get_config
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from utils.utils import set_settings
config = get_config('OurModelConfig')
config.dataset = '201_acc'
set_settings(config)

log_filename, exper_detail = get_experiment_name(config)
plotter = MetricsPlotter(log_filename, config)
log = Logger(log_filename, exper_detail, plotter, config)
datamodule = DataModule(config)

<module 'module.name' from '/home/rtx4090/code/python/current/LightNAS/configs/OurModelConfig.py'> OurModelConfig
✅ All __pycache__ folders removed
|2025-08-26 13:49:07| {
     'ablation': 0, 'ac_loss': True, 'bs': 16
     'classification': False, 'continue_train': False, 'd_model': 100
     'dataset': 201_acc, 'debug': False, 'decay': 0.0001
     'device': cuda, 'dst_dataset': data/nasbench201/pkl/desktop-gpu-gtx-1080ti-fp32.pkl, 'embed_type': nerf
     'epochs': 600, 'eval_set': True, 'f': /run/user/1000/jupyter/runtime/kernel-v3adaa5158a9a8a049120fe198b64322efdce975d3.json
     'ffn_method': ffn, 'gcn_method': gcn, 'hyper_search': False
     'input_size': 192, 'log': <utils.exp_logger.Logger object at 0x710a7b6e2390>, 'logger': zyx
     'loss_func': MSELoss, 'lr': 0.001, 'model': ours
     'monitor_metric': NMAE, 'multires_p': 32, 'multires_r': 32
     'multires_x': 32, 'norm_method': batch, 'num_heads': 4
     'num_layers': 3, 'optim': Adam, 'path': ./data
     'patience': 50, 'pre

100%|██████████| 13909/13909 [00:03<00:00, 3897.50it/s]

|2025-08-26 13:49:11| Train_length : 764 Valid_length : 611 Test_length : 13909


In [12]:
data = datamodule.train_data 
data

{'key': [[0, 2, 1, 0, 4, 3],
  [3, 1, 1, 3, 3, 4],
  [1, 4, 1, 3, 2, 4],
  [4, 4, 3, 3, 3, 2],
  [4, 3, 3, 1, 0, 0],
  [1, 4, 2, 1, 0, 0],
  [4, 3, 2, 3, 3, 1],
  [3, 1, 1, 2, 3, 4],
  [3, 4, 3, 3, 2, 4],
  [1, 0, 3, 0, 3, 0],
  [1, 3, 1, 3, 0, 3],
  [0, 3, 0, 2, 4, 0],
  [1, 1, 3, 2, 0, 1],
  [2, 4, 2, 0, 1, 1],
  [2, 4, 0, 1, 3, 2],
  [3, 3, 3, 4, 1, 1],
  [0, 2, 2, 3, 2, 1],
  [3, 3, 0, 0, 0, 1],
  [3, 1, 2, 0, 1, 2],
  [4, 0, 0, 0, 3, 2],
  [4, 0, 0, 2, 3, 2],
  [4, 2, 1, 3, 0, 3],
  [2, 2, 3, 2, 3, 0],
  [3, 4, 4, 3, 2, 2],
  [2, 4, 0, 1, 3, 1],
  [4, 3, 2, 4, 1, 1],
  [3, 2, 3, 2, 1, 0],
  [2, 4, 4, 1, 0, 2],
  [0, 3, 1, 1, 1, 0],
  [3, 2, 3, 0, 3, 0],
  [3, 3, 3, 2, 3, 1],
  [1, 4, 1, 0, 2, 2],
  [1, 3, 0, 1, 2, 1],
  [1, 0, 3, 0, 2, 1],
  [4, 4, 1, 0, 1, 0],
  [1, 1, 0, 4, 2, 2],
  [1, 0, 1, 4, 3, 2],
  [1, 2, 4, 0, 4, 0],
  [4, 4, 1, 4, 4, 4],
  [3, 3, 3, 3, 3, 0],
  [2, 0, 1, 3, 0, 4],
  [3, 4, 0, 2, 3, 3],
  [2, 2, 1, 1, 4, 2],
  [1, 3, 3, 4, 1, 2],
  [1, 0, 1, 2, 3, 0],
  [

In [18]:
from baselines.narformer import tokenizer
from data_process.create_latency import get_arch_str_from_arch_vector, info2mat
idx = 0
key = data['key'][idx]
arch_str = get_arch_str_from_arch_vector(key)             # 架构向量转字符串
adj_mat, ops_idx = info2mat(arch_str)                     # 得到邻接矩阵与操作
tokens = tokenizer(ops_idx, adj_mat, 32, 32, 32, 'nerf')  # 得到token表示
N = len(ops_idx)

In [ ]:
len(ops_idx)

(8,)

In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
class GraphIdentifierInjector(nn.Module):
    """
    为图数据添加各种结构标识符的工具类
    支持: rand_node_id, orf_node_id, lap_node_id, type_id
    """
    def __init__(
        self,
        node_feat_dim,          # 节点特征维度
        d_model,                # 最终特征维度
        rand_node_id=False,     # 是否使用随机节点ID
        rand_node_id_dim=64,    # 随机节点ID维度
        orf_node_id=True,      # 是否使用正交随机节点ID
        orf_node_id_dim=64,     # 正交随机节点ID维度
        lap_node_id=False,      # 是否使用拉普拉斯特征向量ID
        lap_node_id_k=8,        # 拉普拉斯特征向量数量
        lap_sign_flip=False,    # 拉普拉斯特征向量随机符号翻转
        type_id=False           # 是否使用类型ID（区分节点/边）
    ):
        super().__init__()
        self.node_feat_dim = node_feat_dim
        self.d_model = d_model
        self.lap_sign_flip = lap_sign_flip
        
        # 标识符开关
        self.rand_node_id = rand_node_id
        self.orf_node_id = orf_node_id
        self.lap_node_id = lap_node_id
        self.type_id = type_id
        
        # 计算输入特征总维度（原始特征 + 各类标识符）
        input_dim = node_feat_dim
        if lap_node_id:
            input_dim += lap_node_id_k
        if rand_node_id:
            self.rand_encoder = nn.Linear(2 * rand_node_id_dim, d_model, bias=False)
        if orf_node_id:
            self.orf_encoder = nn.Linear(2 * orf_node_id_dim, d_model, bias=False)
        if type_id:
            self.type_embed = nn.Embedding(2, d_model)  # 0:节点, 1:边
        
        # 节点特征投影层（融合原始特征与拉普拉斯ID）
        self.node_proj = nn.Linear(input_dim, d_model)
        
        # 拉普拉斯特征向量ID相关参数
        if lap_node_id:
            self.lap_node_id_k = lap_node_id_k

    def _compute_laplacian_feats(self, adj):
        """计算拉普拉斯特征向量标识符 [B, n, k]"""
        B, n, _ = adj.shape
        lap_feats = []
        for b in range(B):
            # 计算归一化拉普拉斯矩阵
            A = adj[b].float()
            deg = A.sum(1).clamp(min=1e-12)
            D_inv_sqrt = torch.diag(deg.pow(-0.5))
            L = torch.eye(n, device=adj.device) - D_inv_sqrt @ A @ D_inv_sqrt
            
            # 求解特征值和特征向量（取前k个最小特征值对应的特征向量）
            eigvals, eigvecs = torch.linalg.eigh(L)
            k = min(self.lap_node_id_k, n)
            lap_feat = eigvecs[:, :k]  # [n, k]
            
            # 填充到指定维度
            if k < self.lap_node_id_k:
                lap_feat = F.pad(lap_feat, (0, self.lap_node_id_k - k))
            
            lap_feats.append(lap_feat)
        
        lap_feats = torch.stack(lap_feats, dim=0)  # [B, n, k]
        
        # 随机符号翻转（增强鲁棒性）
        if self.lap_sign_flip and self.training:
            sign_flip = torch.randint(0, 2, lap_feats.shape[:2], device=adj.device) * 2 - 1  # [B, n]
            lap_feats = lap_feats * sign_flip.unsqueeze(-1)
        
        return lap_feats

    def _generate_rand_node_ids(self, node_num, device):
        """生成随机节点ID [sum(node_num), dim]"""
        total_nodes = sum(node_num)
        rand_ids = torch.randn(total_nodes, self.rand_node_id_dim, device=device)
        return F.normalize(rand_ids, p=2, dim=1)  # L2归一化保证正交性

    def _generate_orf_node_ids(self, node_num, max_n, device):
        """生成正交随机节点ID [sum(node_num), dim]"""
        B = len(node_num)
        # 生成高斯正交矩阵
        orf_matrix = self._gaussian_orthogonal_random_matrix(B, max_n, device)  # [B, max_n, max_n]
        
        # 提取有效节点的ID
        orf_ids = []
        ptr = 0
        for b in range(B):
            n = node_num[b]
            orf_ids.append(orf_matrix[b, :n, :n])  # [n, n]
            ptr += n
        
        orf_ids = torch.cat(orf_ids, dim=0)  # [sum(n), n]
        
        # 填充到指定维度
        if self.orf_node_id_dim > max_n:
            orf_ids = F.pad(orf_ids, (0, self.orf_node_id_dim - max_n), value=0.0)
        else:
            orf_ids = orf_ids[:, :self.orf_node_id_dim]
        
        return orf_ids

    @staticmethod
    def _gaussian_orthogonal_random_matrix(B, n, device):
        """生成批量化的高斯正交随机矩阵 [B, n, n]"""
        matrix = torch.randn(B, n, n, device=device)
        q, _ = torch.linalg.qr(matrix)  # QR分解获取正交矩阵
        return q

    def _get_node_mask(self, node_num, device):
        """生成节点掩码 [B, max_n]"""
        B = len(node_num)
        max_n = max(node_num)
        node_mask = torch.zeros(B, max_n, dtype=torch.bool, device=device)
        for b in range(B):
            node_mask[b, :node_num[b]] = True
        return node_mask

    def forward(self, adj, node_feats, edge_index=None):
        """
        为图数据添加各种标识符
        :param adj: 邻接矩阵 [B, n, n]
        :param node_feats: 节点特征 [B, n, c_node]
        :param edge_index: 边索引 [2, E] (可选，用于边标识符)
        :return: 
            node_tokens: 带标识符的节点token [B, n, d_model]
            edge_tokens: 带标识符的边token [B, e, d_model] (若提供edge_index)
        """
        B, n, _ = adj.shape
        device = adj.device
        node_num = [n] * B  # 假设批量中每个图的节点数都是n（可根据实际情况修改）
        node_mask = self._get_node_mask(node_num, device)
        
        # 1. 基础特征拼接（原始特征 + 拉普拉斯ID）
        input_feats = node_feats
        if self.lap_node_id:
            lap_feats = self._compute_laplacian_feats(adj)  # [B, n, k]
            input_feats = torch.cat([input_feats, lap_feats], dim=-1)  # [B, n, c_node + k]
        
        # 2. 节点投影（基础特征 -> d_model）
        node_tokens = self.node_proj(input_feats)  # [B, n, d_model]
        
        # 3. 添加随机节点ID标识符
        if self.rand_node_id:
            rand_ids = self._generate_rand_node_ids(node_num, device)  # [sum(n), dim]
            # 这里简化处理：直接将随机ID投影后加到节点特征（复杂场景可参考边索引关联逻辑）
            rand_embed = self.rand_encoder(torch.cat([rand_ids, rand_ids], dim=-1))  # [sum(n), d_model]
            ptr = 0
            for b in range(B):
                node_tokens[b, :node_num[b]] += rand_embed[ptr:ptr+node_num[b]]
                ptr += node_num[b]
        
        # 4. 添加正交随机节点ID标识符
        if self.orf_node_id:
            max_n = max(node_num)
            orf_ids = self._generate_orf_node_ids(node_num, max_n, device)  # [sum(n), dim]
            orf_embed = self.orf_encoder(torch.cat([orf_ids, orf_ids], dim=-1))  # [sum(n), d_model]
            ptr = 0
            for b in range(B):
                node_tokens[b, :node_num[b]] += orf_embed[ptr:ptr+node_num[b]]
                ptr += node_num[b]
        
        # 5. 添加类型ID（节点类型为0）
        if self.type_id:
            node_type = torch.zeros(n, dtype=torch.long, device=device)  # 节点类型标记为0
            node_tokens += self.type_embed(node_type).unsqueeze(0)  # [B, n, d_model]
        
        # 处理边标识符（如果提供edge_index）
        edge_tokens = None
        if edge_index is not None and (self.rand_node_id or self.orf_node_id or self.type_id):
            # 这里简化实现，实际可根据需要扩展
            edge_src, edge_dst = edge_index
            edge_tokens = torch.zeros(len(edge_src), self.d_model, device=device)
            
            # 边类型ID（类型为1）
            if self.type_id:
                edge_type = torch.ones(len(edge_src), dtype=torch.long, device=device)
                edge_tokens += self.type_embed(edge_type)
        
        return node_tokens, edge_tokens

# ====== 模拟一个 batch ======
B = 2       # batch size
N = 5       # 每个图 5 个节点
C = 16      # 原始节点特征维度

# 1. 构造邻接矩阵 (无向简单图)
adj = torch.zeros(B, N, N)
for b in range(B):
    for i in range(N-1):
        adj[b, i, i+1] = 1
        adj[b, i+1, i] = 1   # 对称化

# 2. 构造节点特征
node_feats = torch.randn(B, N, C)

# 3. 构造边索引 (示例：取第0个图的边)
edge_index = torch.tensor([
    [0, 1, 2],
    [1, 2, 3]
])  # [2, E]，表示 (0->1, 1->2, 2->3)

# ====== 使用 GraphIdentifierInjector ======
injector = GraphIdentifierInjector(
    node_feat_dim=C,
    d_model=32,           # 输出 token 维度
    rand_node_id=True,
    rand_node_id_dim=8,
    orf_node_id=True,
    orf_node_id_dim=8,
    lap_node_id=True,
    lap_node_id_k=4,
    lap_sign_flip=True,
    type_id=True
)

node_tokens, edge_tokens = injector(adj, node_feats, edge_index)

print("邻接矩阵 adj:", adj.shape)
print("节点特征 node_feats:", node_feats.shape)
print("输出 node_tokens:", node_tokens.shape)
print("输出 edge_tokens:", edge_tokens.shape if edge_tokens is not None else None)

AttributeError: 'GraphIdentifierInjector' object has no attribute 'rand_node_id_dim'

In [1]:
# Experiment Settings, logger, plotter
from exp.exp_model import Model
import utils
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from utils.utils import set_settings
from utils.exp_config import get_config
from run_train import get_experiment_name
from data_provider.data_center import DataModule

config = get_config('OurModelConfig')
# config = get_config('FlopsConfig')
# config = get_config('MacConfig')
# config = get_config('LSTMConfig')
# config = get_config('GRUConfig')
# config = get_config('BRPNASConfig')
# config = get_config('GATConfig')
# config = get_config('NarFormerConfig')
# config = get_config('NarFormer2Config')
# config = get_config('NNformerConfig')
set_settings(config)

if config.dataset == 'nnlqp':
    config.input_size = 29
    if config.model == 'narformer':
        config.input_size = 1216
        config.graph_d_model = 960
        config.d_model = 1216
elif config.dataset == '101_acc':
    config.bs = 1024
        
log_filename, exper_detail = get_experiment_name(config)
plotter = MetricsPlotter(log_filename, config)
log = Logger(log_filename, exper_detail, plotter, config)

runid = 0
config.runid = runid
utils.utils.set_seed(config.seed + runid)
datamodule = DataModule(config)
model = Model(config)

/home/rtx4090/anaconda3/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


<module 'module.name' from '/home/rtx4090/code/python/current/LightNAS/configs/OurModelConfig.py'> OurModelConfig
✅ All __pycache__ folders removed
|2025-09-05 15:30:48| {
     'ablation': 0, 'ac_loss': True, 'bs': 256
     'classification': False, 'continue_train': False, 'd_model': 192
     'dataset': 201_acc, 'debug': False, 'decay': 0.0001
     'device': cuda, 'epochs': 1500, 'eval_set': True
     'f': /run/user/1000/jupyter/runtime/kernel-v31338a34f36b5bba3f820c9698ddb582b37dcfd12.json, 'hyper_search': False, 'input_size': 192
     'log': <utils.exp_logger.Logger object at 0x755ebf94da90>, 'logger': zyx, 'loss_func': MSELoss
     'lp_d_model': 16, 'lr': 0.001, 'model': ours
     'monitor_metric': KendallTau, 'monitor_reverse': True, 'num_heads': 4
     'num_layers': 2, 'op_encoder': embedding, 'optim': Adam
     'path': ./data, 'patience': 150, 'predict_target': accuracy
     'rank_loss': True, 'record': True, 'retrain': True
     'rounds': 5, 'runid': 0, 'scaler_method': minmax
 

In [2]:
one_batch = next(iter(datamodule.train_loader))

In [3]:
len(one_batch)

4

In [4]:
adj, features, P, y = one_batch
adj.shape

torch.Size([256, 8, 8])

In [27]:
import torch 
import torch.nn as nn
lap_dim = 16
d_model = 60 
c_node = 60


B, n, _ = adj.shape
edge_proj = nn.Linear(1 + 2 * lap_dim, d_model)
type_embed = nn.Embedding(2, d_model)
node_proj = nn.Linear(c_node + 2 * lap_dim, d_model)
graph_tok = nn.Parameter(torch.randn(1, 1, d_model))

graph_tok = graph_tok.expand(B, -1, -1)  # [B,1,d]
seq = torch.cat([graph_tok] + tokens, dim=1)  # [B, 1+n(+max_m), d]


tokens = []

edge_tok_per_b = [None] * B
edge_len = torch.zeros(B, dtype=torch.long, device=adj.device)
for b in range(B):
    Ab = (adj[b] > 0)
    Ab = Ab.clone()
    Ab.fill_diagonal_(False)
    src, dst = torch.triu(Ab, diagonal=1).nonzero(as_tuple=True)
    m = src.numel()
    edge_len[b] = m
    if m == 0:
        continue
    Pu, Pv = P[b, src], P[b, dst]
    edge_input = torch.cat([torch.ones(m, 1, device=adj.device), Pu, Pv], dim=-1)  # [m,1+2*lap]
    etok = edge_proj(edge_input).unsqueeze(0)                                  # [1,m,d]
    etok = etok + type_embed(torch.ones(m, dtype=torch.long, device=adj.device)).unsqueeze(0)
    edge_tok_per_b[b] = etok

max_m = int(edge_len.max().item() if edge_len.numel() else 0)
if max_m > 0:
    padded = torch.zeros(B, max_m, node_proj.out_features, device=adj.device)
    pad_mask_edges = torch.ones(B, max_m, dtype=torch.bool, device=adj.device)
    for b, etok in enumerate(edge_tok_per_b):
        if etok is None:
            continue
        L = etok.size(1)
        padded[b, :L] = etok[0]
        pad_mask_edges[b, :L] = False
    tokens.append(padded)
else:
    pad_mask_edges = torch.zeros(B, 0, dtype=torch.bool, device=adj.device)
    
graph_tok = graph_tok.expand(B, -1, -1)  # [B,1,d]
seq = torch.cat([graph_tok] + tokens, dim=1)  # [B, 1+n(+max_m), d]

pad_mask = torch.cat([
    torch.zeros(B, 1 + n, dtype=torch.bool, device=adj.device),
    pad_mask_edges
], dim=1)

In [6]:
adj.shape

torch.Size([256, 8, 8])

In [ ]:
import numpy as np
import networkx as nx

def graph_info_from_adj(adj: np.ndarray, directed: bool=False, weighted: bool=False, weight_name: str="weight"):
    """
    adj: n x n 邻接矩阵（numpy）
    directed: 是否按有向图处理
    weighted: True 则从 adj 读取权重；False 则视为无权
    weight_name: 边权重属性名
    """
    n = adj.shape[0]
    assert adj.shape[0] == adj.shape[1], "adj 必须是方阵"
    
    # 构图
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    G.add_nodes_from(range(n))

    if weighted:
        # 允许稀疏权重；忽略对角线
        rows, cols = np.where(adj != 0)
        for i, j in zip(rows, cols):
            if i == j: 
                continue
            w = float(adj[i, j])
            # 无向图避免重复加双边
            if (not directed) and (j < i):
                continue
            G.add_edge(int(i), int(j), **{weight_name: w})
    else:
        rows, cols = np.where(adj != 0)
        for i, j in zip(rows, cols):
            if i == j:
                continue
            if (not directed) and (j < i):
                continue
            G.add_edge(int(i), int(j))

    # 基本统计
    n_nodes = G.number_of_nodes()
    n_edges  = G.number_of_edges()
    deg = dict(G.degree()) if not directed else {"in": dict(G.in_degree()), "out": dict(G.out_degree())}
    neighbors = {u: list(G.neighbors(u)) for u in G.nodes()}

    # 连通性
    if directed:
        weak_components = list(nx.weakly_connected_components(G))
        strong_components = list(nx.strongly_connected_components(G))
        is_weakly_connected  = (len(weak_components) == 1)
        is_strongly_connected = (len(strong_components) == 1)
    else:
        components = list(nx.connected_components(G))
        is_connected = (len(components) == 1)

    # 最短路（无权时用 hop 数；加权时用给定权重）
    weight = weight_name if weighted else None
    # 对分量分别计算最短路，避免非连通报错
    def comp_metrics(H):
        # 最短路长度字典（all-pairs）
        dist = dict(nx.all_pairs_dijkstra_path_length(H, weight=weight)) if weighted \
               else dict(nx.all_pairs_shortest_path_length(H))
        # 转成矩阵（分量节点次序）
        nodes = list(H.nodes())
        idx = {u:i for i,u in enumerate(nodes)}
        D = np.full((len(nodes), len(nodes)), np.inf, dtype=float)
        for u in nodes:
            D[idx[u], idx[u]] = 0.0
            for v, d in dist[u].items():
                D[idx[u], idx[v]] = float(d)

        # 只在有路径的对上取平均
        finite = D[np.isfinite(D)]
        avg_path_len = float(finite[finite>0].mean()) if finite.size and np.any((D>0)&np.isfinite(D)) else np.nan
        # 直径/半径/离心率（仅连通子图上有定义）
        ecc = nx.eccentricity(H, sp=dict(dist)) if len(H) > 0 and nx.is_connected(H.to_undirected()) else None
        diameter = int(nx.diameter(H, e=ecc)) if ecc is not None else None
        radius   = int(nx.radius(H, e=ecc))   if ecc is not None else None
        return {
            "nodes": nodes,
            "D": D,
            "avg_path_length": avg_path_len,
            "eccentricity": ecc,
            "diameter": diameter,
            "radius": radius,
        }

    # 分量逐个指标
    if directed:
        comps_for_metrics = [G.subgraph(c).copy() for c in weak_components]  # 用弱连通分量衡量
    else:
        comps_for_metrics = [G.subgraph(c).copy() for c in components]
    per_component = [comp_metrics(H) for H in comps_for_metrics]

    # 聚类系数 / 三角形
    if directed:
        # 将有向图视为无向来衡量这些无向指标
        Gu = G.to_undirected()
        clustering = nx.clustering(Gu, weight=weight)
        transitivity = nx.transitivity(Gu)
        triangles_node = nx.triangles(Gu)  # 每节点参与的三角数
        triangles_total = int(sum(triangles_node.values()) // 3)
    else:
        clustering = nx.clustering(G, weight=weight)
        transitivity = nx.transitivity(G)
        triangles_node = nx.triangles(G)
        triangles_total = int(sum(triangles_node.values()) // 3)

    # k-core 核数（结构角色）
    if directed:
        Gu = G.to_undirected()
        core_number = nx.core_number(Gu)
        max_core = max(core_number.values()) if core_number else 0
    else:
        core_number = nx.core_number(G) if n_nodes > 0 else {}
        max_core = max(core_number.values()) if core_number else 0

    # 层/level（BFS）：默认以“度数最高”的节点做根
    if directed:
        root = max(G.degree, key=lambda kv: kv[1])[0] if n_nodes else None
        # BFS 用无向视角更稳妥（层级更像“距离”）
        parents = nx.bfs_tree(G.to_undirected(), root) if root is not None else None
    else:
        root = max(G.degree, key=lambda kv: kv[1])[0] if n_nodes else None
        parents = nx.bfs_tree(G, root) if root is not None else None
    level = None
    if parents is not None:
        level = dict(nx.single_source_shortest_path_length(parents, root))  # 到根的跳数层级

    out = {
        "n": n_nodes,
        "m": n_edges,
        "degree": deg,
        "neighbors": neighbors,
        "connected": (is_connected if not directed else None),
        "weakly_connected": (is_weakly_connected if directed else None),
        "strongly_connected": (is_strongly_connected if directed else None),
        "num_components": len(comps_for_metrics),
        "component_sizes": [len(H["nodes"]) for H in per_component],
        "per_component_paths": per_component,  # 含 D/avg_path/直径/半径/离心率
        "clustering_node": clustering,
        "clustering_global_transitivity": transitivity,
        "triangles_node": triangles_node,
        "triangles_total": triangles_total,
        "core_number": core_number,
        "max_core": max_core,
        "bfs_root": root,
        "bfs_level": level,  # 到根的层级/深度
    }
    return out


# ==== 示例 ====
# 一个小例子：无向图邻接矩阵
A = np.array([
    [0,1,1,0,0],
    [1,0,1,0,0],
    [1,1,0,1,0],
    [0,0,1,0,1],
    [0,0,0,1,0],
], dtype=int)

info = graph_info_from_adj(A, directed=False, weighted=False)
print("n, m:", info["n"], info["m"])
print("is_connected:", info["connected"])
print("component_sizes:", info["component_sizes"])
print("avg_path_length (comp0):", info["per_component_paths"][0]["avg_path_length"])
print("diameter (comp0):", info["per_component_paths"][0]["diameter"])
print("global clustering (transitivity):", info["clustering_global_transitivity"])
print("max_core:", info["max_core"])
print("bfs_root / level size:", info["bfs_root"], len(info["bfs_level"]) if info["bfs_level"] else 0)

n, m: 5 5
is_connected: True
component_sizes: [5]
avg_path_length (comp0): 1.7
diameter (comp0): 3
global clustering (transitivity): 0.5
max_core: 2
bfs_root / level size: 2 5


In [9]:
import numpy as np
import networkx as nx

def _build_graph_from_adj(adj: np.ndarray, directed: bool, weighted: bool, weight_name="weight"):
    n = adj.shape[0]
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    G.add_nodes_from(range(n))
    rows, cols = np.where(adj != 0)
    for i, j in zip(rows, cols):
        if i == j:
            continue
        if not directed and j < i:
            continue
        if weighted:
            G.add_edge(int(i), int(j), **{weight_name: float(adj[i, j])})
        else:
            G.add_edge(int(i), int(j))
    return G

def _component_metrics(H: nx.Graph, weighted: bool, weight_key: str):
    # 最短路/距离矩阵
    if weighted:
        dist = dict(nx.all_pairs_dijkstra_path_length(H, weight=weight_key))
    else:
        dist = dict(nx.all_pairs_shortest_path_length(H))

    nodes = list(H.nodes())
    idx = {u: i for i, u in enumerate(nodes)}
    D = np.full((len(nodes), len(nodes)), np.inf, dtype=float)
    for u in nodes:
        D[idx[u], idx[u]] = 0.0
        for v, d in dist[u].items():
            D[idx[u], idx[v]] = float(d)

    # 仅对有限且>0 的距离做均值
    mask = (D > 0) & np.isfinite(D)
    avg_path = float(D[mask].mean()) if np.any(mask) else np.nan

    # 直径/半径/离心率（连通组件才有）
    ecc = None
    diameter = None
    radius = None
    if len(H) > 1:
        Hu = H.to_undirected() if H.is_directed() else H
        if nx.is_connected(Hu):
            try:
                ecc = nx.eccentricity(Hu)
                diameter = nx.diameter(Hu, e=ecc)
                radius = nx.radius(Hu, e=ecc)
            except Exception:
                pass

    # 中心/边缘
    center = None
    periphery = None
    if diameter is not None:
        try:
            Hu = H.to_undirected() if H.is_directed() else H
            center = nx.center(Hu, e=ecc)
            periphery = nx.periphery(Hu, e=ecc)
        except Exception:
            pass

    # 效率
    global_eff = nx.global_efficiency(H.to_undirected() if H.is_directed() else H)

    return {
        "nodes": nodes,
        "D": D,  # 组件内距离矩阵（无穷表示不连通）
        "avg_path_length": avg_path,
        "eccentricity": ecc,
        "diameter": diameter,
        "radius": radius,
        "center": center,
        "periphery": periphery,
        "global_efficiency": global_eff,
    }

def graph_topology(
    adj: np.ndarray,
    directed: bool = False,
    weighted: bool = False,
    weight_name: str = "weight",
    heavy: bool = True,        # 打开更重的计算（谱特征/连通度/介数等）
    bfs_root: int | None = None
):
    """
    仅基于邻接矩阵提取拓扑信息（不使用节点/边属性）。
    adj: n×n numpy 矩阵
    directed: 是否按有向图处理
    weighted: 是否读取权重
    heavy: 是否开启耗时指标
    bfs_root: BFS 层级的根；None 时使用“度最大”的节点
    """
    assert adj.ndim == 2 and adj.shape[0] == adj.shape[1], "adj 必须是方阵"
    G = _build_graph_from_adj(adj, directed=directed, weighted=weighted, weight_name=weight_name)
    n = G.number_of_nodes()
    m = G.number_of_edges()

    # 基本统计
    density = nx.density(G)
    is_directed = G.is_directed()
    is_weighted = weighted

    # 度信息
    if not is_directed:
        deg = dict(G.degree())
        deg_vals = np.array(list(deg.values()), dtype=float)
        deg_stats = {
            "min": float(deg_vals.min()) if len(deg_vals) else 0.0,
            "max": float(deg_vals.max()) if len(deg_vals) else 0.0,
            "mean": float(deg_vals.mean()) if len(deg_vals) else 0.0,
            "std": float(deg_vals.std()) if len(deg_vals) else 0.0,
        }
    else:
        indeg = dict(G.in_degree())
        outdeg = dict(G.out_degree())
        indeg_vals = np.array(list(indeg.values()), dtype=float)
        outdeg_vals = np.array(list(outdeg.values()), dtype=float)
        deg = {"in": indeg, "out": outdeg}
        deg_stats = {
            "in": {
                "min": float(indeg_vals.min()) if len(indeg_vals) else 0.0,
                "max": float(indeg_vals.max()) if len(indeg_vals) else 0.0,
                "mean": float(indeg_vals.mean()) if len(indeg_vals) else 0.0,
                "std": float(indeg_vals.std()) if len(indeg_vals) else 0.0,
            },
            "out": {
                "min": float(outdeg_vals.min()) if len(outdeg_vals) else 0.0,
                "max": float(outdeg_vals.max()) if len(outdeg_vals) else 0.0,
                "mean": float(outdeg_vals.mean()) if len(outdeg_vals) else 0.0,
                "std": float(outdeg_vals.std()) if len(outdeg_vals) else 0.0,
            }
        }

    # 邻居表
    neighbors = {u: list(G.neighbors(u)) for u in G.nodes()}

    # 连通性
    if is_directed:
        weak_components = list(nx.weakly_connected_components(G))
        strong_components = list(nx.strongly_connected_components(G))
        is_weakly_connected = (len(weak_components) == 1)
        is_strongly_connected = (len(strong_components) == 1)
        comps = [G.subgraph(c).copy() for c in weak_components]  # 用弱连通做距离统计
    else:
        components = list(nx.connected_components(G))
        is_connected = (len(components) == 1)
        comps = [G.subgraph(c).copy() for c in components]

    # 距离/路径（按组件）
    per_component = [_component_metrics(H, weighted=weighted, weight_key=weight_name) for H in comps]

    # 聚类/三角形
    if is_directed:
        Gu = G.to_undirected()
        clustering_node = nx.clustering(Gu, weight=weight_name if weighted else None)
        transitivity = nx.transitivity(Gu)
        triangles_node = nx.triangles(Gu)
    else:
        clustering_node = nx.clustering(G, weight=weight_name if weighted else None)
        transitivity = nx.transitivity(G)
        triangles_node = nx.triangles(G)
    triangles_total = int(sum(triangles_node.values()) // 3)

    # k-core / 最大核
    if is_directed:
        Gu = G.to_undirected()
        core_number = nx.core_number(Gu) if n > 0 else {}
    else:
        core_number = nx.core_number(G) if n > 0 else {}
    max_core = max(core_number.values()) if core_number else 0

    # 桥/割点（仅无向图定义明确）
    bridges = list(nx.bridges(G)) if not is_directed else None
    articulation_points = list(nx.articulation_points(G)) if not is_directed else None

    # 丰富：同配性、富俱乐部系数、局部/全局效率
    assort = None
    try:
        assort = nx.degree_assortativity_coefficient(G.to_undirected() if is_directed else G)
    except Exception:
        pass

    rich_club = None
    try:
        rich_club = nx.rich_club_coefficient(G.to_undirected() if is_directed else G, normalized=False)
    except Exception:
        pass

    local_eff = None
    try:
        local_eff = nx.local_efficiency(G.to_undirected() if is_directed else G)
    except Exception:
        pass

    # BFS 层级（层/level）
    if bfs_root is None:
        if is_directed:
            root = max(G.degree, key=lambda kv: kv[1])[0] if n else None
        else:
            root = max(G.degree, key=lambda kv: kv[1])[0] if n else None
    else:
        root = bfs_root if bfs_root in G.nodes else None

    bfs_level = None
    if root is not None:
        tree_base = (G.to_undirected() if is_directed else G)
        parents = nx.bfs_tree(tree_base, root)
        bfs_level = dict(nx.single_source_shortest_path_length(parents, root))

    # 有向专属：互惠性/入出度相关性/PageRank
    reciprocity = nx.reciprocity(G) if is_directed else None
    pagerank = None
    if is_directed and heavy and n > 0:
        try:
            pagerank = nx.pagerank(G, alpha=0.85, max_iter=200)
        except Exception:
            pass

    # 各种中心性（可能较重）
    centrality = {}
    if heavy and n > 0:
        Hu = G.to_undirected() if is_directed else G
        # 度中心性（快）
        centrality["degree"] = nx.degree_centrality(Hu)

        # 介数/负载中心性（较重）
        try:
            centrality["betweenness"] = nx.betweenness_centrality(Hu, normalized=True)
        except Exception:
            pass
        try:
            centrality["load"] = nx.load_centrality(Hu, normalized=True)
        except Exception:
            pass

        # 近/谐波中心性（快）
        try:
            centrality["closeness"] = nx.closeness_centrality(Hu)
        except Exception:
            pass
        try:
            centrality["harmonic"] = nx.harmonic_centrality(Hu)
        except Exception:
            pass

        # 特征向量中心性（可能不收敛，给容错）
        try:
            centrality["eigenvector"] = nx.eigenvector_centrality_numpy(Hu)  # 用 numpy 求特征向量稳定
        except Exception:
            pass

        # Katz（挑小 alpha）
        try:
            centrality["katz"] = nx.katz_centrality_numpy(Hu, alpha=0.01, beta=1.0)
        except Exception:
            pass

        # current-flow 类中心性（需要连通）
        try:
            if nx.is_connected(Hu):
                centrality["current_flow_closeness"] = nx.current_flow_closeness_centrality(Hu)
                centrality["current_flow_betweenness"] = nx.current_flow_betweenness_centrality(Hu, normalized=True)
        except Exception:
            pass

    # 谱特征 / 生成树数 / 代数连通度 等（较重）
    spectral = {}
    if heavy and n > 0:
        Hu = G.to_undirected() if is_directed else G
        A = nx.to_numpy_array(Hu, weight=weight_name if weighted else None)
        L = nx.laplacian_matrix(Hu, weight=weight_name if weighted else None).toarray()

        # 邻接谱
        try:
            vals_A = np.linalg.eigvals(A)
            vals_A = np.sort(np.real(vals_A))[::-1]
            spectral["adjacency_eigs_topk"] = vals_A[:min(10, len(vals_A))].tolist()
            spectral["spectral_radius"] = float(np.max(np.abs(vals_A))) if len(vals_A) else 0.0
        except Exception:
            pass

        # 拉普拉斯谱
        try:
            vals_L = np.linalg.eigvals(L)
            vals_L = np.sort(np.real(vals_L))
            spectral["laplacian_eigs_smallk"] = vals_L[:min(10, len(vals_L))].tolist()
            # 代数连通度（Fiedler 值）: 第二小特征值（连通时 >0）
            if len(vals_L) >= 2:
                spectral["algebraic_connectivity"] = float(vals_L[1])
            else:
                spectral["algebraic_connectivity"] = None
        except Exception:
            pass

        # 生成树数量（Kirchhoff 矩阵树定理：拉普拉斯特征值的非零部分乘积 / n）
        try:
            eps = 1e-12
            nonzero = [lam for lam in np.real(vals_L) if lam > eps]
            if len(nonzero) >= (n - 1):  # 对于连通图应有 n-1 个非零值
                tau = float(np.prod(nonzero) / n)
            else:
                tau = 0.0
            spectral["num_spanning_trees_est"] = tau
        except Exception:
            pass

        # 可通信性 / Estrada 指数
        try:
            C = nx.communicability_exp(Hu)  # 字典的字典
            estrada = nx.estrada_index(Hu)
            spectral["estrada_index"] = float(estrada)
            # 可选：节点可通信性求和（节点层面的“可达/互联程度”）
            comm_sum = {u: float(sum(C[u].values())) for u in C}
            spectral["communicability_sum"] = comm_sum
        except Exception:
            pass

    # 连通度（节点/边，较重）
    connectivity = {}
    if heavy and n > 1:
        Hu = G.to_undirected() if is_directed else G
        try:
            connectivity["node_connectivity"] = nx.node_connectivity(Hu)
        except Exception:
            pass
        try:
            connectivity["edge_connectivity"] = nx.edge_connectivity(Hu)
        except Exception:
            pass

    # 汇总
    out = {
        "n": n,
        "m": m,
        "density": density,
        "directed": is_directed,
        "weighted": is_weighted,
        "degree": deg,
        "degree_stats": deg_stats,
        "neighbors": neighbors,
        "assortativity": assort,
        "rich_club_coefficient": rich_club,  # dict: k -> φ(k)
        "local_efficiency": local_eff,
        "clustering_node": clustering_node,
        "clustering_global_transitivity": transitivity,
        "triangles_node": triangles_node,
        "triangles_total": triangles_total,
        "core_number": core_number,
        "max_core": max_core,
        "bridges": bridges,
        "articulation_points": articulation_points,
        "bfs_root": root,
        "bfs_level": bfs_level,
        "per_component": per_component,  # 每个分量含: D/avg_path/直径/半径/中心/边缘/效率
        "connectivity": connectivity,    # 节点/边连通度（heavy）
        "centrality": centrality,        # 多种中心性（heavy）
        "spectral": spectral,            # 谱特征/生成树/可通信性（heavy）
        "reciprocity": reciprocity,      # 有向图互惠性
        "pagerank": pagerank,            # 有向图 PageRank（heavy）
        "is_connected": (is_connected if not is_directed else None),
        "is_weakly_connected": (is_weakly_connected if is_directed else None),
        "is_strongly_connected": (is_strongly_connected if is_directed else None),
        "num_components": len(per_component),
        "component_sizes": [len(c["nodes"]) for c in per_component],
    }
    return out


# ===== 示例 =====
if __name__ == "__main__":
    # 无向、无权例子
    A = np.array([
        [0,1,1,0,0,0],
        [1,0,1,0,0,0],
        [1,1,0,1,0,0],
        [0,0,1,0,1,0],
        [0,0,0,1,0,1],
        [0,0,0,0,1,0],
    ], dtype=float)

    info = graph_topology(A, directed=False, weighted=False, heavy=True)
    print("n,m,density:", info["n"], info["m"], info["density"])
    print("connected:", info["is_connected"], "components:", info["num_components"])
    print("avg_path(comp0):", info["per_component"][0]["avg_path_length"])
    print("diameter(comp0):", info["per_component"][0]["diameter"])
    print("transitivity:", info["clustering_global_transitivity"])
    print("max_core:", info["max_core"])
    print("node_connectivity:", info["connectivity"].get("node_connectivity"))
    print("spectral_radius:", info["spectral"].get("spectral_radius"))
    print("algebraic_connectivity:", info["spectral"].get("algebraic_connectivity"))
    print("num_spanning_trees_est:", info["spectral"].get("num_spanning_trees_est"))

n,m,density: 6 6 0.4
connected: True components: 1
avg_path(comp0): 2.066666666666667
diameter(comp0): 4
transitivity: 0.42857142857142855
max_core: 2
node_connectivity: 1
spectral_radius: 2.2283282159466364
algebraic_connectivity: 0.32486912943335383
num_spanning_trees_est: 3.0000000000000013


X shape: (5, 30)
列段信息:
  deg                          -> [0, 1) 维数=1
  log1p_deg                    -> [1, 2) 维数=1
  kcore                        -> [2, 3) 维数=1
  triangles                    -> [3, 4) 维数=1
  clustering                   -> [4, 5) 维数=1
  pagerank                     -> [5, 6) 维数=1
  closeness                    -> [6, 7) 维数=1
  harmonic                     -> [7, 8) 维数=1
  eigenvector                  -> [8, 9) 维数=1
  rwpe_K8                      -> [9, 17) 维数=8
  lpe_d5                       -> [17, 22) 维数=5
  anchor_buckets_R5_D6         -> [22, 30) 维数=8


In [2]:
import torch
import numpy as np
def lap_eig(dense_adj, number_of_nodes, in_degree):
    """
    Graph positional encoding v/ Laplacian eigenvectors
    https://github.com/DevinKreuzer/SAN/blob/main/data/molecules.py
    """
    dense_adj = dense_adj.detach().cpu().float().numpy()
    in_degree = in_degree.detach().cpu().float().numpy()

    # Laplacian
    A = dense_adj
    N = np.diag(in_degree.clip(1) ** -0.5)
    L = np.eye(number_of_nodes) - N @ A @ N

    # (sorted) eigenvectors with numpy
    EigVal, EigVec = np.linalg.eigh(L)

    # for eigval, take abs because numpy sometimes computes the first eigenvalue approaching 0 from the negative
    eigvec = torch.from_numpy(EigVec).float()  # [N, N (channels)]
    eigval = torch.from_numpy(np.sort(np.abs(np.real(EigVal)))).float()  # [N (channels),]
    return eigvec, eigval  # [N, N (channels)]  [N (channels),]


def get_pe_2d(edge2_indices: torch.LongTensor, edge12_indices: torch.LongTensor, n_node: int, n_edge12: int,
              half_pos_enc_dim=128):
    assert n_node <= half_pos_enc_dim
    device = edge12_indices.device
    # edge2_indices: [2,?]: the indices for the 2-edge of the graph
    # edge12_indices: [|E|, 2] the indices for all 1-edge and 2-edge of the graph from the Batch data object 

    dense_adj = torch.zeros([n_node, n_node], dtype=torch.bool, device=device)
    dense_adj[edge2_indices[0, :], edge2_indices[1, :]] = True
    in_degree = dense_adj.long().sum(dim=1).view(-1)

    EigVec, EigVal = lap_eig(dense_adj, n_node, in_degree)  # EigVec: [N, N] 
    node_pe = torch.zeros(n_node, half_pos_enc_dim).to(device)  # [N, half_pos_enc_dim]
    node_pe[:, :n_node] = EigVec
    E = edge12_indices.shape[0]
    all_edges_pe = torch.zeros([E, 2 * half_pos_enc_dim]).to(device)
    all_edges_pe[:n_edge12, :half_pos_enc_dim] = torch.index_select(node_pe, 0, edge12_indices[:n_edge12, 0])
    all_edges_pe[:n_edge12, half_pos_enc_dim:] = torch.index_select(node_pe, 0, edge12_indices[:n_edge12, 1])

    return all_edges_pe.unsqueeze(0)  # [1, E, 2*half_pos_enc_dim]